In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

import datetime as dt

%matplotlib inline

import panel as pn

pn.extension('plotly')

In [2]:
def check_types(col):
    temp_df = DataFrame()
    temp_df['types'] = [type(x) for x in col]
    return temp_df['types'].unique()

In [3]:
url_ld =r'C:\Users\ASUS\Documents\GitHub\semirara-hr-analytics\ld_data.csv'
url_hc = r'C:\Users\ASUS\Documents\GitHub\semirara-hr-analytics\cleaned2.csv'
df = pd.read_csv(url_hc,sep=',')
df_ld = pd.read_csv(url_ld,sep=',')

In [4]:
str_date = lambda x: np.nan if x != x else dt.datetime.strptime(x,'%m/%d/%Y')

df['date_hire']=df.date_hire.apply(str_date)
df['date_prob']=df.date_prob.apply(str_date)
# df['date_reg']=df.date_reg.apply(str_date)
df['date_sep']=df.date_sep.apply(str_date)
df['date_birth']=df.date_birth.apply(str_date)

date_now = dt.datetime.now()

In [5]:
df_ld['COMPANY 2'].replace({'DPC':'SEM-Calaca Power Corporation'},inplace=True)

In [6]:
df_ld['TRAINING CATEGORY']=df_ld['TRAINING CATEGORY'].apply(lambda x: x.upper())
df_ld['TRAINING CATEGORY'].replace({'B':'BEHAVIORAL','L':'LEADERSHIP','PTD':'PROFESSIONAL AND TECHNICAL DEVELOPMENT',
                                    'QMS': 'QUALITY MANAGEMENT SYSTEM'},inplace=True)

In [7]:
ld_mo_dict={'JAN':'JANUARY','FEB':'FEBRUARY','MAR':'MARCH','APR':'APRIL',
            'AUG':'AUGUST','SEPT':'SEPTEMBER','OCT':'OCTOBER','NOV':'NOVEMBER','DEC':'DECEMBER'}

df_ld['Month']=df_ld['Month'].apply(lambda x: x.upper())
df_ld['Month'].replace(ld_mo_dict,inplace=True)

ld_mo2_dict={'JANUARY':dt.datetime(2019,1,1),'FEBRUARY':dt.datetime(2019,2,1),'MARCH':dt.datetime(2019,3,1),'APRIL':dt.datetime(2019,4,1),
             'MAY':dt.datetime(2019,5,1),'JUNE':dt.datetime(2019,6,1),'JULY':dt.datetime(2019,7,1),'AUGUST':dt.datetime(2019,8,1),
             'SEPTEMBER':dt.datetime(2019,9,1),'OCTOBER':dt.datetime(2019,10,1),'NOVEMBER':dt.datetime(2019,11,1),'DECEMBER':dt.datetime(2019,12,1)}

df_ld['Month'].replace(ld_mo2_dict,inplace=True)

In [8]:
df_ld['TRAINING START'] = [x.split()[0] for x in df_ld['TRAINING START']]
df_ld['TRAINING START'] = df_ld['TRAINING START'].apply(lambda x: np.nan if x != x else dt.datetime.strptime(x,'%m/%d/%Y' ))
df_ld['TRAINING END'] = [x.split()[0] for x in df_ld['TRAINING END']]
df_ld['TRAINING END'] = df_ld['TRAINING END'].apply(lambda x: np.nan if x != x else dt.datetime.strptime(x,'%m/%d/%Y' ))

In [9]:
def get_days(days):
    start,end = days
    if end - start == dt.timedelta(0):
        return dt.timedelta(1)
    else:
        return end - start
    
df_ld['training_duration2']=df_ld[['TRAINING START','TRAINING END']].apply(get_days,1)

In [10]:
import plotly.graph_objects as go

title ='# HR Dashboard'
sub1 = '## By Age'
sub2 = '## By Gender'
sub3 = '## L&D Training Hours'

comps = df['company'].unique().tolist()
aggs = ['Sum','Average']

comp = pn.widgets.Select(name='Company', options=comps, margin=(0, 5, 5, 5))
agg = pn.widgets.RadioBoxGroup(name='Aggregation', options=aggs)



def get_df_hc(comp):
    dft = df[(df['company']==comp) & (df.active_stat == 'Active')]
    return dft

def get_df_ld(comp,agg):
    df = df_ld[df_ld['COMPANY 2']==comp]
    
    if agg == 'Sum':
        return df.groupby('Month')['TOTAL HOURS'].sum().reset_index()
    elif agg == 'Average':
        return df.groupby('Month')['TOTAL HOURS'].mean().reset_index()


@pn.depends(comp.param.value)
def get_plot_age(comp):
    dft = get_df_hc(comp)
    dft = dft.groupby('generation')['emp_num'].count().reset_index()
    pie_plot = go.Pie(labels=dft.generation, 
            values=dft.emp_num, hole=.50)
    
    return go.Figure(data=pie_plot)

@pn.depends(comp.param.value)
def get_plot_gender(comp):
    dft = get_df_hc(comp)
    dft = dft.groupby('gender')['emp_num'].count().reset_index()
    pie_plot = go.Pie(labels=dft.gender, 
            values=dft.emp_num, hole=.50)
    
    return go.Figure(data=pie_plot)

@pn.depends(comp.param.value)
def get_plot_class(comp):
    dft = get_df_hc(comp)
    dft = dft.groupby('emp_class2')['emp_num'].count().reset_index()
    pie_plot = go.Pie(labels=dft.emp_class2, 
            values=dft.emp_num, hole=.50)
    return go.Figure(data=pie_plot)

@pn.depends(comp.param.value,agg.param.value)
def get_plot_ld(comp,agg):
    
    df = get_df_ld(comp,agg)
    scat_plot = go.Scatter(x=df.Month.apply(lambda x: x.strftime("%Y/%m")), y=df['TOTAL HOURS'], 
                      name = comp, mode='lines+markers')
    fig = go.Figure(data=scat_plot)
    fig.layout.yaxis.update({'title': '{} of Training Hours'.format(agg)})
    return fig


pane_headcount=pn.Row(
    pn.Column(title,comp,sub1, get_plot_age),
    pn.Column('','','','','',sub2, get_plot_gender)
)

pane_ld = pn.Row(
    pn.Column(title,comp,agg,
    sub3,get_plot_ld)
)

pn.Tabs(pane_headcount)

tabs = pn.Tabs(('People Demographics',pane_headcount))

tabs.append(('Learning', pane_ld))

# pane.show()
tabs.servable('Semirara HR Dashboard')

Tabs
    [0] Row
        [0] Column
            [0] Markdown(str)
            [1] Select(margin=(0, 5, 5, 5), name='Company', options=['SEM-Calaca Power Corpora...], value='SEM-Calaca Power C...)
            [2] Markdown(str)
            [3] ParamFunction(function)
        [1] Column
            [0] Markdown(str)
            [1] Markdown(str)
            [2] Markdown(str)
            [3] Markdown(str)
            [4] Markdown(str)
            [5] Markdown(str)
            [6] ParamFunction(function)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] Select(margin=(0, 5, 5, 5), name='Company', options=['SEM-Calaca Power Corpora...], value='SEM-Calaca Power C...)
            [2] RadioBoxGroup(name='Aggregation', options=['Sum', 'Average'], value='Sum')
            [3] Markdown(str)
            [4] ParamFunction(function)